# Skills and Progressive Disclosure

Skills are reusable capability abstractions that reveal detail only when needed. This example shows an agent that discovers available skills, activates one, and uses its tools.

In [ ]:
from pathlib import Path
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.skills.registry import SkillRegistry
from agentic_patterns.core.skills.tools import list_available_skills, get_skill_instructions

## Setup: Discover Skills

Discovery loads only metadata (name + description). Full instructions stay on disk until needed.

In [ ]:
skills_root = Path("skills-demo")
registry = SkillRegistry()
registry.discover([skills_root])

print("Discovered skills:")
for meta in registry.list_all():
    print(f"  {meta}")

## Define Agent Tools

The agent has two tools:
- `activate_skill`: Load full instructions (progressive disclosure)
- `analyze_code`: A tool provided by the code-review skill

In [ ]:
activated_skills: set[str] = set()

def activate_skill(skill_name: str) -> str:
    """Activate a skill by name to get its full instructions and enable its tools."""
    instructions = get_skill_instructions(registry, skill_name)
    if instructions is None:
        return f"Skill '{skill_name}' not found."
    activated_skills.add(skill_name)
    print(f"[SKILL ACTIVATED: {skill_name}]")
    return instructions

def analyze_code(code: str) -> str:
    """Analyze code for security issues. Requires code-review skill to be activated."""
    if "code-review" not in activated_skills:
        return "Error: You must activate the 'code-review' skill first."
    print(f"[SKILL TOOL CALLED: analyze_code]")
    print(f"  Analyzing: {code[:50]}...")
    # Simulated analysis
    issues = []
    if "eval(" in code:
        issues.append("CRITICAL: eval() with user input allows arbitrary code execution")
    if "exec(" in code:
        issues.append("CRITICAL: exec() with user input allows arbitrary code execution")
    if not issues:
        issues.append("No critical issues found")
    return "\n".join(issues)

## Create the Agent

The system prompt includes the skill catalog. The agent must activate a skill before using its tools.

In [ ]:
skill_catalog = list_available_skills(registry)

system_prompt = f"""You are an assistant with access to skills.

Available skills:
{skill_catalog}

To use a skill:
1. Call activate_skill(skill_name) to load its instructions
2. Read the instructions to understand what tools are available
3. Use the skill's tools (e.g., analyze_code for code-review)

You must activate a skill before using its tools."""

agent = get_agent(
    system_prompt=system_prompt,
    tools=[activate_skill, analyze_code]
)

## Run the Agent

Watch for `[SKILL ACTIVATED]` and `[SKILL TOOL CALLED]` markers in the output.

In [ ]:
activated_skills.clear()

prompt = "Review this Python code for security issues: `data = eval(user_input)`"

agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\n--- Final Response ---\n{agent_run.result.output}")

## What Happened

The agent:
1. Saw the skill catalog (metadata only - cheap)
2. Called `activate_skill("code-review")` to load full instructions
3. Called `analyze_code()` - which only works because the skill was activated
4. Reported findings based on the tool result

The `[SKILL ACTIVATED]` and `[SKILL TOOL CALLED]` markers show progressive disclosure in action: instructions loaded on demand, tools gated behind activation.